# Preprocessing

In [ ]:
import re
import numpy as np

import nltk
from nltk.stem import WordNetLemmatizer
from sklearn.preprocessing import LabelEncoder
from hmmlearn import hmm
from nltk import FreqDist

In [ ]:
#open txt and deleting empty rows
with open("../sample/HP.txt", "r", encoding = "latin-1") as f:
   HP_text = f.read()
type(HP_text)
#HP_text = HP_text.replace("\n", "")
print(HP_text)

In [ ]:
# create a lemmatizer that turns all charachters lower-case, splits it into words and removes all punctuation
# except for end of sentence punctations. 
lemmatizer = WordNetLemmatizer()
def lemmatize(message):
    message = message.lower().replace('.', ' .').replace('!', ' !').replace('?', ' ?')
    #''.join([char for char in message.lower()
                       #if char.isalnum() or char.isspace()])
    words= [lemmatizer.lemmatize(word, pos='v') 
            for word in message.split()]
    punctuations = '"#$%&\'()*+,-/:;<=>@[\\]^_`{|}~'
    table = str.maketrans('', '', punctuations)
    return [w.translate(table) for w in words]

lemmatize(HP_text) 

In [ ]:
HP_words = lemmatize(HP_text) 

In [ ]:
def sentencer(text):
    '''split by . ? !
    filter empty'''
    sentences = re.split(r'[.?!]', text)
    return [sentence for sentence in sentences if len(sentence)]
    
def preprocess(text):
    sentences = sentencer(text)
    return [lemmatize(sentence) for sentence in sentences]

In [ ]:
HP_lemmas = preprocess(HP_text)
HP_words = [word for sentence in HP_lemmas for word in sentence]

# Training

In [ ]:
alphabet = set(HP_words)
le = LabelEncoder()
le.fit(list(alphabet))

seq = le.transform(HP_words)
features = np.fromiter(seq, np.int64)
features = np.atleast_2d(features).T
fd = FreqDist(seq)

model = hmm.MultinomialHMM(n_components=8, init_params='ste')

lengths = [len(lemma) for lemma in HP_lemmas]
model = model.fit(features, lengths)


In [ ]:
def frequencies():
    frequencies = np.fromiter((fd.freq(i) for i in range(len(alphabet))), dtype=np.float64)
    emission_prob = np.stack([frequencies] * 8)

    model = hmm.MultinomialHMM(n_components=8, init_params='st')
    model.emissionprob_ = emission_prob
    return model
model = frequencies().fit(features, lengths)

# Generating

In [ ]:
symbols, _states = model.sample(8)

output = le.inverse_transform(np.squeeze(symbols))

output